Tu es un assistant spécialisé en statistiques et en développement Pyspark et pandas. Tu vas écrire un programme complet, en précisant chaque ligne par un commentaire.

## Requirement

In [ ]:
%pip install imblearn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import homogeneity_score
from sklearn.model_selection import train_test_split  
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
from mlflow.models.signature import infer_signature

In [ ]:
# customization of autologging
mlflow.autolog(
    log_input_examples=False,
    log_model_signatures=True,
    log_models=True,
    disable=False,
    exclusive=False, # Update this property to enable custom logging
    disable_for_unsupported_versions=True,
    silent=True
)

mlflow.set_experiment("orders-clustering-experiment")

In [ ]:
print(f"MLFlow Tracking URI: {mlflow.get_tracking_uri()}")

## Data preparation

In [ ]:
############################
### PRECISEZ VOTRE LOGIN ###
############################
login = "<ICI VOTRE LOGIN>" # login = "GM61"

In [ ]:
orders_path = f"abfss://{login}@onelake.dfs.fabric.microsoft.com/AmazingZoneLH.Lakehouse/Tables/dbo_Order"
orders_df = spark.read.format("delta").load(orders_path)
# orders_df = spark.sql("SELECT * FROM AmazingZoneLH.dbo_Order")

print(orders_df.columns)

In [ ]:
orders_pdf = orders_df \
                .withColumnRenamed("Id", "OrderId") \
                .select("OrderId", "CustomerId", "PickupInStore", "OrderTotal", "AllowStoringCreditCardNumber", "ShippingMethod") \
                .toPandas()

# display(orders_pdf)

#### Order items

In [ ]:
orderitems_path = f"abfss://{login}@onelake.dfs.fabric.microsoft.com/AmazingZoneLH.Lakehouse/Tables/dbo_OrderItem"
orderitem_df = spark.read.format("delta").load(orderitems_path)
# orderitem_df = spark.sql("SELECT * FROM AmazingZoneLH.dbo_OrderItem")
# print(orderitem_df.columns)

In [ ]:
orderitem_pdf = orderitem_df.toPandas()
# display(orderitem_df)

#### Aggregate Order items

In [ ]:
orderDetails_agg_pdf = orderitem_pdf.groupby('OrderId').agg(
    nombre_produits=('ProductId', 'nunique'),     # Nombre de produits commandés (produits uniques)
    prix_unitaire_median=('UnitPriceInclTax', 'median'), # Prix unitaire médian
    quantite_totale=('Quantity', 'sum')          # Quantité totale
).reset_index()

# display(orderDetails_agg_pdf)

#### Golden dataset

In [ ]:
# Agréger les DataFrames sur la colonne 'OrderId'
golden_pdf = orders_pdf.merge(orderDetails_agg_pdf, on='OrderId', how='left')

In [ ]:
# Convertir les colonnes 'OrderTotal' et 'prix_unitaire_median' en type float
golden_pdf['OrderTotal'] = pd.to_numeric(golden_pdf['OrderTotal'], errors='coerce')  
golden_pdf['prix_unitaire_median'] = pd.to_numeric(golden_pdf['prix_unitaire_median'], errors='coerce')  

In [ ]:
display(golden_pdf)

In [ ]:
# Afficher les types de données du DataFrame initial  
print("Types de données du DataFrame initial:")  
print(golden_pdf.dtypes)

In [ ]:
print("Dimensions du DataFrame initial:")
print(golden_pdf.shape)

In [ ]:
print("Valeurs distinctes de ShippingMethod :", golden_pdf['ShippingMethod'].unique())

In [ ]:
# Keep it if you want to save dataframe as a delta lake, parquet table to Tables section of the default lakehouse

golden_df = spark.createDataFrame(golden_pdf)
golden_df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable("dbo_golden_ds")

## Machine Learning

### Préparation des colonnes numériques et catégorielles

In [ ]:
# Retirer les lignes avec des valeurs NaN
clean_golden_pdf = golden_pdf.dropna()
print("Dimensions du DataFrame sans NaN:")
print(clean_golden_pdf.shape)

In [ ]:
kmeans_pdf = clean_golden_pdf

# Convertir les booléens en entiers (0 et 1)
kmeans_pdf['AllowStoringCreditCardNumber'] = kmeans_pdf['AllowStoringCreditCardNumber'].astype(int)

# Encoder les colonnes catégorielles (ShippingMethod)
kmeans_pdf = pd.get_dummies(kmeans_pdf, columns=['ShippingMethod'], drop_first=True)

# Remplacer les espaces par des underscores dans les noms de colonnes
kmeans_pdf.columns = kmeans_pdf.columns.str.replace(' ', '_')

# Séparer les OrderId et CustomerId pour l'analyse ultérieure
order_ids = kmeans_pdf['OrderId']
customer_ids = kmeans_pdf['CustomerId']

# Retirer OrderId et CustomerId pour l'entraînement du modèle
kmeans_pdf = kmeans_pdf.drop(columns=['OrderId', 'CustomerId'])

# Normaliser les données
scaler = StandardScaler()
kmeans_pdf = scaler.fit_transform(kmeans_pdf)

### Détermination du meilleur nombre de clusters

L'inertie mesure la compacité des clusters formés par l'algorithme KMeans. Plus l'inertie est faible, plus les points de données sont proches de leurs centres de clusters respectifs, ce qui indique une meilleure qualité de classification.

Elle correspond à la somme des distances quadratiques des points de données à leur centre de cluster le plus proche.

In [ ]:
# Déterminer le meilleur nombre de clusters avec la méthode du coude
wcss = []
for i in range(1, 11):  
    # kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans = KMeans(n_clusters=i, max_iter=1000, random_state=42)
    kmeans.fit(kmeans_pdf)  
    wcss.append(kmeans.inertia_)  
  
# Tracer le graphique du coude  
plt.figure(figsize=(10, 8))  
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')  
plt.title('Méthode du coude')  
plt.xlabel('Nombre de clusters')  
plt.ylabel('WCSS (Within-Cluster Sum of Squares)')  
plt.show()



### Calcul des KMeans pour le meilleur nombre de classes

In [ ]:
#####################################
### CHOISIR LE NOMBRE DE CLUSTERS ###
#####################################
n_clusters = 4

In [ ]:
# Appliquer KMeans avec le nombre de clusters choisi
try:
    mlflow.end_run()
finally:
    mlflow.set_experiment("fraud-detect-experiment")

with mlflow.start_run():
    # kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=300, n_init=10, random_state=42)
    clusters = kmeans.fit_predict(kmeans_pdf)
    mlflow.log_metric("inertia", round(kmeans.inertia_, 0))
    # TODO: ajouter homogeneity score 
    # homogeneity_score([0, 0, 1, 1], [1, 1, 0, 0])
  
# Ajouter les clusters au DataFrame  
clean_golden_pdf['Cluster'] = clusters

# Remettre les colonnes OrderId et CustomerId
clean_golden_pdf['OrderId'] = order_ids 
clean_golden_pdf['CustomerId'] = customer_ids

### Evaluation de la variance expliquée par les composantes principales

In [ ]:
# customization of autologging
mlflow.autolog(
    log_input_examples=False,
    log_model_signatures=True,
    log_models=True,
    disable=True, # on ne loggue pas l'ACP
    exclusive=False,
    disable_for_unsupported_versions=True,
    silent=True
)

In [ ]:
# Appliquer l'ACP  
pca = PCA() # le paramètre n_components n'est pas renseigné : il est égal au nombre de caractéristiques des données d'entrée.
pca.fit(kmeans_pdf)  
  
# Calculer la variance expliquée cumulée  
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)  
# display(cumulative_variance)
  
# Tracer la variance expliquée cumulée  
plt.figure(figsize=(10, 6))  
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--')  
plt.title('Variance expliquée cumulée par les composantes principales')  
plt.xlabel('Nombre de composantes principales')  
plt.ylabel('Variance expliquée cumulée')  
plt.axhline(y=0.90, color='r', linestyle='-')  
plt.axhline(y=0.95, color='g', linestyle='-')  
plt.show()  
  
# Déterminer le nombre de composantes pour expliquer 90% de la variance  
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1  
print(f"Nombre de composantes pour expliquer 90% de la variance: {n_components_90}")  
  
# Déterminer le nombre de composantes pour expliquer 95% de la variance  
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1  
print(f"Nombre de composantes pour expliquer 95% de la variance: {n_components_95}")  

### Affichage du premier plan de l'ACP

In [ ]:
# Réaliser une ACP pour réduire les dimensions à 2 composantes principales
n_components = 7

pca = PCA(n_components=n_components)  
pca_pdf = pca.fit_transform(kmeans_pdf)
  
# Ajouter les composantes principales au DataFrame
component_columns = [f'PCA{i+1}' for i in range(n_components)]

# Créer un DataFrame avec les composantes principales
pca_pdf = pd.DataFrame(data=pca_pdf, columns=component_columns)

pca_pdf['Cluster'] = kmeans.labels_
pca_pdf['OrderId'] = order_ids

pca_pdf['CustomerId'] = customer_ids

In [ ]:
###########################################
### CHOISIR LES COMPOSANTES PRINCIPALES ###
###########################################
x = 'PCA1'
y = 'PCA2'

# Représentation graphique des clusters  
plt.figure(figsize=(10, 8))  
sns.scatterplot(x=x, y=y, hue='Cluster', data=pca_pdf, palette='viridis', s=100, alpha=0.7)  
plt.title('Représentation des clusters après ACP')  
plt.xlabel('PCA1')  
plt.ylabel('PCA2')  
plt.legend(title='Cluster')  
plt.show()

### Description des classes

In [ ]:
# # Calculer les statistiques descriptives pour chaque cluster  
# cluster_groups = clean_golden_pdf[['OrderTotal', 'nombre_produits', 'prix_unitaire_median', 'quantite_totale', 'Cluster']].groupby('Cluster')  
  
# # Initialiser un dictionnaire pour stocker les métriques  
# cluster_metrics = {}  
  
# # Calculer les métriques pour chaque cluster  
# for cluster, group in cluster_groups:  
#     metrics = {  
#         'mean': group.mean(),  
#         'median': group.median(),  
#         'std': group.std(),  
#         'min': group.min(),  
#         'max': group.max(),  
#         '25%': group.quantile(0.25),  
#         '50%': group.quantile(0.50),  # qui est aussi la médiane  
#         '75%': group.quantile(0.75)  
#     }  
#     cluster_metrics[cluster] = metrics  
  
# # Afficher les métriques pour chaque cluster  
# for cluster, metrics in cluster_metrics.items():  
#     print(f"\nCluster {cluster} Metrics:")  
#     for metric_name, metric_values in metrics.items():  
#         print(f"\n{metric_name.capitalize()}:")  
#         print(metric_values)

In [ ]:
# Grouper par 'Cluster' et calculer la moyenne pour les variables quantitatives
quantitative_vars = ['OrderTotal', 'nombre_produits', 'prix_unitaire_median', 'quantite_totale']

cluster_means = clean_golden_pdf.groupby('Cluster')[quantitative_vars].mean().reset_index()

# Créer des boxplots pour les moyennes des variables quantitatives par cluster
plt.figure(figsize=(15, 10))

for i, var in enumerate(quantitative_vars):
    plt.subplot(3, 2, i + 1)
    sns.boxplot(x='Cluster', y=var, data=clean_golden_pdf)
    plt.title(f'{var} par Cluster')

plt.tight_layout()
plt.show()

## Détection des commandes aberrantes

In [ ]:
# Ajouter la colonne 'Outlier' pour les commandes du cluster identifié
outlier_cluster = 3
clean_golden_pdf['Outlier'] = np.where(clean_golden_pdf['Cluster'] == outlier_cluster, True, False)

print(clean_golden_pdf['Outlier'].value_counts())

In [ ]:
from imblearn.over_sampling import SMOTE


# Supprimer les colonnes inutiles
outliers_pdf = clean_golden_pdf.drop(columns=['CustomerId', 'OrderId', 'Cluster'])
print(outliers_pdf.columns)

# Convertir les colonnes booléennes et object en variables numériques (dummies)
outliers_pdf = pd.get_dummies(outliers_pdf, drop_first=True)
print(outliers_pdf.columns)

# Remplacer les espaces par des underscores dans les noms de colonnes
outliers_pdf.columns = outliers_pdf.columns.str.replace(' ', '_')
print(outliers_pdf.columns)

# Séparer les features et la cible
X = outliers_pdf.drop(columns=['Outlier'])
y = outliers_pdf['Outlier']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print(X_train.columns)

# Compter le nombre d'échantillons dans la classe minoritaire
n_minority_samples = y_train.value_counts().min()

# Définir k_neighbors à une valeur appropriée
k_neighbors = min(n_minority_samples - 1, 5)

# Utiliser SMOTE pour équilibrer les classes dans l'ensemble d'entraînement
smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [ ]:
# customization of autologging
mlflow.autolog(
    log_input_examples=False,
    log_model_signatures=True,
    log_models=True,
    disable=False,
    exclusive=False,
    disable_for_unsupported_versions=True,
    silent=True
)

# Initialiser le client MLflow
from mlflow.tracking import MlflowClient

client = MlflowClient()

mlflow.set_experiment("fraud-detection-experiment")

In [ ]:
with mlflow.start_run():

    ####################################
    ### MODIFIER LES HYPERPARAMETRES ###
    ####################################

    # Entraîner un modèle RandomForest
    model = RandomForestClassifier(
        n_estimators=100,          # Nombre d'arbres dans la forêt
        max_depth=10,              # Profondeur maximale des arbres
        min_samples_split=5,       # Nombre minimum d'échantillons requis pour diviser un nœud
        min_samples_leaf=2,        # Nombre minimum d'échantillons requis pour être à une feuille de nœud
        max_features='sqrt',       # Nombre de caractéristiques à considérer pour la meilleure séparation
        bootstrap=True,            # Si les échantillons sont tirés avec remplacement
        # class_weight='balanced',   # Ajuster les poids des classes pour gérer le déséquilibre
        random_state=42            # Assurer la reproductibilité
    )
    model.fit(X_train_res, y_train_res)

    # Prédire les étiquettes sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric('Accuracy', accuracy)
    print(f'Accuracy: {accuracy:.2f}')

    # Precision
    precision = precision_score(y_test, y_pred, average='weighted')
    mlflow.log_metric('Precision', precision)
    print(f'Precision: {precision:.2f}')

    # Recall
    recall = recall_score(y_test, y_pred, average='weighted')
    mlflow.log_metric('Recall', recall)
    print(f'Recall: {recall:.2f}')

    # F1-Score
    f1 = f1_score(y_test, y_pred, average='weighted')
    mlflow.log_metric('F1-Score', f1)
    print(f'F1-Score: {f1:.2f}')

    # Afficher le rapport de classification et la matrice de confusion
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

    # Enregistrer le modèle
    signature = infer_signature(
        X_train, y_pred
    )

    model_name = "orders-outliers-model"

    mlflow.sklearn.log_model(
            model,
            model_name,
            signature=signature,
            registered_model_name=model_name
    )

In [ ]:
model_versions = client.search_model_versions(f"name='{model_name}'")
model_name

for version in model_versions:
    print("Model Name: {}".format(version.name))
    print("Model Version: {}".format(version.version))
    print("Run ID: {}".format(version.run_id))

# Trouve la version la plus élevée
max_version = max([int(version.version) for version in model_versions])

print(f"La version la plus élevée du modèle '{model_name}' est: {max_version}")

In [ ]:
# Optionnel : Affichage des importances des caractéristiques
importances = model.feature_importances_
feature_names = X.columns
feature_importances = pd.DataFrame({'feature': feature_names, 'importance': importances})

display(feature_importances.sort_values(by='importance', ascending=False))

## Génération de nouvelles données aléatoires pour tester l'inférence

In [ ]:
def generate_new_data(n_samples):

    """
    Générer des nouvelles données aléatoires pour tester l'inférence
    """

    new_data = pd.DataFrame({
        # 'Id': np.arange(1, n_samples + 1),
        # 'CustomerId': np.random.randint(1000, 5000, size=n_samples),
        'PickupInStore': np.random.choice([True, False], size=n_samples),
        'OrderTotal': np.random.uniform(10.0, 1000.0, size=n_samples),
        'AllowStoringCreditCardNumber': np.random.choice([True, False], size=n_samples),
        #  ['Ground' 'Next Day Air' 'Pickup in store' '' None]
        'ShippingMethod': np.random.choice(['Ground', 'Next Day Air', 'Pickup in store'], size=n_samples),
        # 'OrderId': np.random.randint(900000, 999999, size=n_samples),
        'nombre_produits': np.random.uniform(1, 3, size=n_samples),
        'prix_unitaire_median': np.random.uniform(1.0, 100.0, size=n_samples),
        'quantite_totale': np.random.uniform(1, 10, size=n_samples),
    })
    return new_data

In [ ]:
# Générer de nouvelles données
nb_new_data = 30
new_data = generate_new_data(nb_new_data)

display(new_data)

In [ ]:
new_data_OHE = pd.get_dummies(new_data, drop_first=False)
new_data_OHE.columns = new_data_OHE.columns.str.replace(' ', '_')

new_data_OHE_df = spark.createDataFrame(new_data_OHE)
print(new_data_OHE_df.columns)

In [ ]:
from synapse.ml.predict import MLFlowTransformer


model = MLFlowTransformer(
    inputCols=list(new_data_OHE_df.columns),
    outputCol='predictions',
    modelName=model_name,
    modelVersion=max_version
)

predictions = model.transform(new_data_OHE_df)

display(predictions)

In [ ]:
# Enregistrer en tant que table
predictions.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable("dbo_OrderPrediction")

In [ ]:
from pyspark.ml.feature import SQLTransformer 


model_name = model_name
model_version = max_version
features = new_data_OHE_df.columns

# Utilise la méthode join pour créer une chaîne de texte avec les colonnes séparées par une virgule
features_str = ", ".join(features)

sqlt = SQLTransformer().setStatement( 
    f"SELECT PREDICT('{model_name}/{model_version}', {','.join(features)}) as predictions, {features_str} FROM __THIS__")

display(sqlt.transform(new_data_OHE_df))